# Modelling

In this notebook we will finally start working on getting a model to predict who may win in a game of chess given our variables. 

## Loading the data and libraries

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

from sklearn.linear_model import LogisticRegression

In [10]:
train = pd.read_pickle('train_df.pkl')
test = pd.read_pickle('test_df.pkl')

In [19]:
train.head()

,rated,victory_status_mate,victory_status_outoftime,victory_status_resign,turns,white_rating,black_rating,opening_ply,time_allotted,increment,...,Counts_name_e87,Counts_name_e88,Counts_name_e90,Counts_name_e91,Counts_name_e92,Counts_name_e94,Counts_name_e95,Counts_name_e97,Counts_name_e98,winner_label
2858,0,1,0,0,0.163828,-0.976846,-0.378900,-0.648908,-0.221724,-0.372712,...,0,0,0,0,0,0,0,0,0,0
7948,1,0,0,1,0.790929,2.243676,1.463170,1.852692,-0.397451,0.351493,...,0,0,0,0,0,0,0,0,0,2
15696,1,0,0,1,1.298583,-0.218264,-0.113780,-0.291536,-0.280300,-0.010609,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,1,-1.538306,-0.635484,1.783380,0.065835,0.071153,1.799901,...,0,0,0,0,0,0,0,0,0,0
14894,1,0,0,1,0.343000,-0.011379,0.020502,0.065835,-0.221724,-0.372712,...,0,0,0,0,0,0,0,0,0,0


Recall that the winner_label is the target variable, and values of 0 indicates that the players who chose black won, 1 indicates a draw, and 2 indicates that players who chose white won.

In [80]:
y = train['winner_label']
X = train.drop(columns = 'winner_label')

In [81]:
X.head()

,rated,victory_status_mate,victory_status_outoftime,victory_status_resign,turns,white_rating,black_rating,opening_ply,time_allotted,increment,...,Counts_name_e81,Counts_name_e87,Counts_name_e88,Counts_name_e90,Counts_name_e91,Counts_name_e92,Counts_name_e94,Counts_name_e95,Counts_name_e97,Counts_name_e98
2858,0,1,0,0,0.163828,-0.976846,-0.378900,-0.648908,-0.221724,-0.372712,...,0,0,0,0,0,0,0,0,0,0
7948,1,0,0,1,0.790929,2.243676,1.463170,1.852692,-0.397451,0.351493,...,0,0,0,0,0,0,0,0,0,0
15696,1,0,0,1,1.298583,-0.218264,-0.113780,-0.291536,-0.280300,-0.010609,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,1,-1.538306,-0.635484,1.783380,0.065835,0.071153,1.799901,...,0,0,0,0,0,0,0,0,0,0
14894,1,0,0,1,0.343000,-0.011379,0.020502,0.065835,-0.221724,-0.372712,...,0,0,0,0,0,0,0,0,0,0


In [85]:
y_test = test['winner_label']
X_test = test.drop(columns = 'winner_label')

## Logistic Regression

In [82]:
#if don't use solver='liblinear', we get an error about str not being able to decode, 
#probably related to the victory status columns
lr = LogisticRegression(solver='liblinear', random_state=42)

lr.fit(X, y)

LogisticRegression(random_state=42, solver='liblinear')

In [84]:
lr.score(X, y)

0.8231477605692759

In [90]:
y_pred = lr.predict(X_test)

In [86]:
lr.score(X_test, y_test)

0.7625026156099602

In [91]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.7625026156099602